In [1]:
!pip install -U altair

Requirement already up-to-date: altair in c:\users\16177\anaconda3\envs\pyvizenv\lib\site-packages (4.1.0)


In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import json
from pathlib import Path
import numpy as np

import altair as alt

In [3]:
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=["Unnamed: 0"])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [6]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.drop(crypto_df[crypto_df["IsTrading"] == False].index)
crypto_df.head()

crypto_df["IsTrading"].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [7]:
# Keep only cryptocurrencies with a working algorithm
crypto_df.drop(crypto_df[crypto_df["Algorithm"] == ""].index, inplace=True)

In [8]:
# Remove the "IsTrading" column
crypto_df.drop(columns=["IsTrading"], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [9]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna(how="any",axis=0)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [10]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df["TotalCoinsMined"].replace("", np.nan, inplace=True)
crypto_df["TotalCoinsMined"].dropna(inplace=True)
crypto_df["TotalCoinsMined"].isnull().sum()

crypto_df.drop(crypto_df[crypto_df["TotalCoinsMined"] <= 0].index, inplace=True)

crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [11]:
# Drop rows where there are 'N/A' text values
crypto_df.dropna(inplace=True)

crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [12]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name = pd.DataFrame(data=crypto_df["CoinName"], index=crypto_df.index)

# coins_name = crypto_df[["CoinName"]]
coin_name.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [13]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns=["CoinName"], inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [14]:
# Create dummy variables for text features
crypto_df["TotalCoinSupply"] = crypto_df["TotalCoinSupply"].apply(pd.to_numeric)
crypto_df.dtypes
X = pd.get_dummies(crypto_df)
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Standardize data
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

In [16]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

# Get two principal components for the data.
crypto_pca = pca.fit_transform(X_scaled)

In [17]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index
)
pca_df.head(10)

,PC 1,PC 2,PC 3
42,-0.331633,1.050460,-0.533304
404,-0.314976,1.050582,-0.533535
1337,2.307696,1.607949,-0.595656
BTC,-0.145176,-1.287660,0.179741
ETH,-0.155106,-1.998909,0.331007
LTC,-0.163361,-1.108308,-0.018367
DASH,-0.399003,1.213091,-0.467171
XMR,-0.145379,-2.171625,0.387181
ETC,-0.153550,-1.999020,0.330980
ZEC,-0.119142,-2.099433,0.523321


In [18]:
inertia = []
k = list(range(1, 11))


# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=1)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

alt.Chart(df_elbow).mark_line(size=3).encode(
    alt.X("k"),
    alt.Y("inertia"),
    tooltip=["k", "inertia"]

).interactive()

alt.Chart(...)

In [19]:
model = KMeans(n_clusters=5, random_state=1)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pca_df["Class"] = model.labels_
pca_df.head()

,PC 1,PC 2,PC 3,Class
42,-0.331633,1.050460,-0.533304,2
404,-0.314976,1.050582,-0.533535,2
1337,2.307696,1.607949,-0.595656,4
BTC,-0.145176,-1.287660,0.179741,0
ETH,-0.155106,-1.998909,0.331007,0


In [21]:
# Creating requested cluster_df dataframe by joining multiple dataframes previously defined. 
clustered_df = crypto_df.join(pca_df).join(coin_name)
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.331633,1.050460,-0.533304,2,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.314976,1.050582,-0.533535,2,404Coin
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.307696,1.607949,-0.595656,4,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.145176,-1.287660,0.179741,0,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.155106,-1.998909,0.331007,0,Ethereum
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.163361,-1.108308,-0.018367,0,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.399003,1.213091,-0.467171,2,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.145379,-2.171625,0.387181,0,Monero
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.153550,-1.999020,0.330980,0,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.119142,-2.099433,0.523321,0,ZCash


In [22]:
alt.Chart(clustered_df).mark_circle(size=60).encode(
    alt.X("PC 1"),
    alt.Y("PC 2"),
    alt.Color("Class"),
    alt.OpacityValue(4),
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)

In [23]:
# Table with tradable cryptos
display(clustered_df)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.331633,1.050460,-0.533304,2,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.314976,1.050582,-0.533535,2,404Coin
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.307696,1.607949,-0.595656,4,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.145176,-1.287660,0.179741,0,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.155106,-1.998909,0.331007,0,Ethereum
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.163361,-1.108308,-0.018367,0,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.399003,1.213091,-0.467171,2,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.145379,-2.171625,0.387181,0,Monero
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.153550,-1.999020,0.330980,0,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.119142,-2.099433,0.523321,0,ZCash


In [24]:
# Print the total number of tradable cryptocurrencies
tradable_crypto = clustered_df["CoinName"].value_counts().sum()
print(f"Total number of tradable cryptocurrencies: {tradable_crypto}")

Total number of tradable cryptocurrencies: 532


In [25]:
scaler = MinMaxScaler()

clustered_df[["TotalCoinsMined", "TotalCoinSupply"]] = scaler.fit_transform(
    clustered_df[["TotalCoinsMined", "TotalCoinSupply"]])

clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,0.000000,4.200000e-11,-0.331633,1.050460,-0.533304,2,42 Coin
404,Scrypt,PoW/PoS,0.001066,5.320000e-04,-0.314976,1.050582,-0.533535,2,404Coin
1337,X13,PoW/PoS,0.029576,3.141593e-01,2.307696,1.607949,-0.595656,4,EliteCoin
BTC,SHA-256,PoW,0.000018,2.100000e-05,-0.145176,-1.287660,0.179741,0,Bitcoin
ETH,Ethash,PoW,0.000109,0.000000e+00,-0.155106,-1.998909,0.331007,0,Ethereum


In [26]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
alt.Chart(clustered_df).mark_circle(size=60).encode(
    alt.X("TotalCoinsMined"),
    alt.Y("TotalCoinSupply"),
    alt.Color("Class"),
    alt.OpacityValue(4),
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)

In [27]:
scaler = StandardScaler()

clustered_df[["TotalCoinsMined", "TotalCoinSupply"]] = scaler.fit_transform(
    clustered_df[["TotalCoinsMined", "TotalCoinSupply"]])

In [28]:
alt.Chart(clustered_df).mark_circle(size=60).encode(
    alt.X("TotalCoinsMined"),
    alt.Y("TotalCoinSupply"),
    alt.Color("Class"),
    alt.OpacityValue(4),
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)